In [9]:
!pip install gradio

In [10]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
df = pd.read_excel("winemag-data-130k-v2.xlsx")
df = df.dropna(subset=['description', 'price', 'country', 'variety']).reset_index(drop=True)


In [12]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])

In [32]:
def recommend_wines(description, variety, country, max_price):
    try:
        new_vec = tfidf.transform([description])
        cosine_sim = cosine_similarity(new_vec, tfidf_matrix).flatten()
        sorted_indices = cosine_sim.argsort()[::-1]

        recomment = []
        for idx in sorted_indices:
            wine = df.iloc[idx]

            if max_price and wine['price'] > max_price:
                continue
            if country and country.lower() != 'none' and wine['country'].lower() != country.lower():
                continue
            if variety and variety.lower() != 'none' and wine['variety'].lower() != variety.lower():
                continue

            recomment.append({
                "Title": wine['title'],
                "Variety": wine['variety'],
                "Country": wine['country'],
                "Price ($)": wine['price'],
                "Points": wine['points'],
                "Description": wine['description']
            })

            if len(recomment) == 5:
                break

        if not recomment:
            return pd.DataFrame([{"Title": "No matching wines found. Try different filters or a broader description."}])

        return pd.DataFrame(recomment)

    except Exception as e:
        return pd.DataFrame([{"Title": f"Error: {str(e)}"}])

In [33]:
varieties = ["None"] + sorted(df['variety'].dropna().unique())
countries = ["None"] + sorted(df['country'].dropna().unique())

In [35]:
with gr.Blocks(title="🍷 WineMate - Smart Wine Recommender") as demo:
    gr.Markdown("""
    # 🍷 **WineMate**
    _Your personalized wine finder based on taste, variety, country, and price._
    """)

    with gr.Row():
        desc_input = gr.Textbox(label="🔍 Describe your wine preference", placeholder="e.g. fruity, dry red with hints of oak")

    with gr.Row():
        variety_input = gr.Dropdown(varieties, label="🍇 Select Variety", allow_custom_value=True)
        country_input = gr.Dropdown(countries, label="🌍 Select Country", allow_custom_value=True)
        price_input = gr.Slider(5, 500, step=1, value=50, label="💸 Max Price ($)")

    output = gr.Dataframe(label="📋 Top 5 Wine Matches")

    gr.Button("🔎 Get Recommendations").click(
        fn=recommend_wines,
        inputs=[desc_input, variety_input, country_input, price_input],
        outputs=output
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://761df8d80586bb2868.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
